In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [3]:
import torch.utils.data as Data
import os
if not os.path.isdir('saved_models_{}'.format('bigan')):
    os.mkdir('saved_models_{}'.format('bigan'))
f = open('saved_models_{}/log_collapse1.txt'.format('bigan'), mode='w')
import torch.utils.data as Data

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K
import os
import matplotlib.pyplot as plt

import numpy as np

class BIGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # Build the encoder
        self.encoder = self.build_encoder()

        # The part of the bigan that trains the discriminator and encoder
        self.discriminator.trainable = False

        # Generate image from sampled noise
        z = Input(shape=(self.latent_dim, ))
        img_ = self.generator(z)

        # Encode image
        img = Input(shape=self.img_shape)
        z_ = self.encoder(img)

        # Latent -> img is fake, and img -> latent is valid
        fake = self.discriminator([z, img_])
        valid = self.discriminator([z_, img])

        # Set up and compile the combined model
        # Trains generator to fool the discriminator
        self.bigan_generator = Model([z, img], [fake, valid])
        self.bigan_generator.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
            optimizer=optimizer)


    def build_encoder(self):
        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.latent_dim))

        model.summary()

        img = Input(shape=self.img_shape)
        z = model(img)

        return Model(img, z)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        z = Input(shape=(self.latent_dim,))
        gen_img = model(z)

        return Model(z, gen_img)

    def build_discriminator(self):

        z = Input(shape=(self.latent_dim, ))
        img = Input(shape=self.img_shape)
        d_in = concatenate([z, Flatten()(img)])

        model = Dense(1024)(d_in)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        validity = Dense(1, activation="sigmoid")(model)

        return Model([z, img], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test, y_test) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_test = X_test / 127.5 - 1.
        X_test = np.expand_dims(X_test, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                z = np.random.normal(size=(batch_size, self.latent_dim))
                imgs_ = self.generator.predict(z)

                # Select a random batch of images and encode
                # idx = np.random.randint(0, X_train.shape[0], batch_size)
                # imgs = X_train[idx]
                z_ = self.encoder.predict(image_batch)

                # Train the discriminator (img -> z is valid, z -> img is fake)
                d_loss_real = self.discriminator.train_on_batch([z_, image_batch], valid)
                d_loss_fake = self.discriminator.train_on_batch([z, imgs_], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (z -> img is valid and img -> z is is invalid)
                g_loss = self.bigan_generator.train_on_batch([z, image_batch], [valid, fake])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch,global_step, d_loss[0],
                                                                                   100 * d_loss[1], g_loss[0]))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')
        

if __name__ == '__main__':
    bigan = BIGAN()
    bigan.train(epochs=30, batch_size=64, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 512)               51712     
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 512)               2048      
_________________________________________________________________
dense_16 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 512)               2048      
_________________________________________________________________
dense_17 (Dense)             (None, 784)               402192    
__________

epoch:0 step:83 [D loss: 0.468622, acc: 83.59%] [G loss: 13.275326]
epoch:0 step:84 [D loss: 0.426609, acc: 89.06%] [G loss: 15.853758]
epoch:0 step:85 [D loss: 0.283986, acc: 87.50%] [G loss: 14.888668]
epoch:0 step:86 [D loss: 0.327465, acc: 85.16%] [G loss: 14.879415]
epoch:0 step:87 [D loss: 0.303412, acc: 85.16%] [G loss: 16.639004]
epoch:0 step:88 [D loss: 0.432522, acc: 82.81%] [G loss: 14.719597]
epoch:0 step:89 [D loss: 0.254990, acc: 88.28%] [G loss: 12.733261]
epoch:0 step:90 [D loss: 0.438938, acc: 82.03%] [G loss: 17.461018]
epoch:0 step:91 [D loss: 0.481008, acc: 83.59%] [G loss: 12.678728]
epoch:0 step:92 [D loss: 0.479376, acc: 82.81%] [G loss: 12.122629]
epoch:0 step:93 [D loss: 0.226798, acc: 90.62%] [G loss: 11.484596]
epoch:0 step:94 [D loss: 0.433246, acc: 80.47%] [G loss: 12.704674]
epoch:0 step:95 [D loss: 0.230730, acc: 90.62%] [G loss: 10.656030]
epoch:0 step:96 [D loss: 0.413457, acc: 85.16%] [G loss: 13.527205]
epoch:0 step:97 [D loss: 0.204907, acc: 89.06%] 

##############
[ 7.1299072   7.59974359 13.25367035 10.80824     8.84389209 11.98333983
 11.27914699  9.82118836 10.00487991  9.29102348]
##########
epoch:0 step:201 [D loss: 0.180940, acc: 93.75%] [G loss: 6.005892]
epoch:0 step:202 [D loss: 0.552187, acc: 75.78%] [G loss: 7.067804]
epoch:0 step:203 [D loss: 0.213903, acc: 88.28%] [G loss: 5.527696]
epoch:0 step:204 [D loss: 0.187607, acc: 96.09%] [G loss: 4.904421]
epoch:0 step:205 [D loss: 0.332821, acc: 79.69%] [G loss: 6.451403]
epoch:0 step:206 [D loss: 0.197755, acc: 92.19%] [G loss: 6.173525]
epoch:0 step:207 [D loss: 0.486503, acc: 74.22%] [G loss: 6.836219]
epoch:0 step:208 [D loss: 0.170131, acc: 92.97%] [G loss: 5.150594]
epoch:0 step:209 [D loss: 0.344044, acc: 88.28%] [G loss: 5.528923]
epoch:0 step:210 [D loss: 0.303972, acc: 87.50%] [G loss: 6.065314]
epoch:0 step:211 [D loss: 0.173801, acc: 95.31%] [G loss: 4.721954]
epoch:0 step:212 [D loss: 0.475361, acc: 80.47%] [G loss: 6.364856]
epoch:0 step:213 [D loss: 0.186712,

epoch:0 step:324 [D loss: 0.421119, acc: 82.03%] [G loss: 4.410597]
epoch:0 step:325 [D loss: 0.288865, acc: 91.41%] [G loss: 4.378838]
epoch:0 step:326 [D loss: 0.522106, acc: 74.22%] [G loss: 4.342999]
epoch:0 step:327 [D loss: 0.217289, acc: 92.97%] [G loss: 3.877490]
epoch:0 step:328 [D loss: 0.599822, acc: 63.28%] [G loss: 5.617480]
epoch:0 step:329 [D loss: 0.261669, acc: 91.41%] [G loss: 3.976883]
epoch:0 step:330 [D loss: 0.443512, acc: 82.81%] [G loss: 4.571805]
epoch:0 step:331 [D loss: 0.241081, acc: 94.53%] [G loss: 3.698300]
epoch:0 step:332 [D loss: 0.516601, acc: 75.00%] [G loss: 4.290234]
epoch:0 step:333 [D loss: 0.295089, acc: 88.28%] [G loss: 3.912292]
epoch:0 step:334 [D loss: 0.485350, acc: 75.78%] [G loss: 3.970228]
epoch:0 step:335 [D loss: 0.288798, acc: 89.84%] [G loss: 4.584590]
epoch:0 step:336 [D loss: 0.351034, acc: 90.62%] [G loss: 4.446581]
epoch:0 step:337 [D loss: 0.429524, acc: 78.91%] [G loss: 4.139856]
epoch:0 step:338 [D loss: 0.421664, acc: 85.16%]

epoch:0 step:445 [D loss: 0.382835, acc: 85.16%] [G loss: 4.226974]
epoch:0 step:446 [D loss: 0.503469, acc: 73.44%] [G loss: 3.935754]
epoch:0 step:447 [D loss: 0.350972, acc: 90.62%] [G loss: 4.344460]
epoch:0 step:448 [D loss: 0.520063, acc: 73.44%] [G loss: 4.157565]
epoch:0 step:449 [D loss: 0.318261, acc: 89.84%] [G loss: 4.322023]
epoch:0 step:450 [D loss: 0.313123, acc: 87.50%] [G loss: 4.573184]
epoch:0 step:451 [D loss: 0.541551, acc: 74.22%] [G loss: 4.108429]
epoch:0 step:452 [D loss: 0.446274, acc: 77.34%] [G loss: 4.438254]
epoch:0 step:453 [D loss: 0.430661, acc: 83.59%] [G loss: 3.999035]
epoch:0 step:454 [D loss: 0.549417, acc: 70.31%] [G loss: 4.105881]
epoch:0 step:455 [D loss: 0.340521, acc: 89.06%] [G loss: 3.996634]
epoch:0 step:456 [D loss: 0.526692, acc: 72.66%] [G loss: 3.727266]
epoch:0 step:457 [D loss: 0.417185, acc: 79.69%] [G loss: 4.077831]
epoch:0 step:458 [D loss: 0.459851, acc: 81.25%] [G loss: 4.071331]
epoch:0 step:459 [D loss: 0.314928, acc: 88.28%]

...

epoch:29 step:27550 [D loss: 0.679866, acc: 58.59%] [G loss: 1.735748]
epoch:29 step:27551 [D loss: 0.640677, acc: 63.28%] [G loss: 1.864820]
epoch:29 step:27552 [D loss: 0.619984, acc: 63.28%] [G loss: 2.045815]
epoch:29 step:27553 [D loss: 0.617433, acc: 66.41%] [G loss: 1.986179]
epoch:29 step:27554 [D loss: 0.608069, acc: 70.31%] [G loss: 1.954936]
epoch:29 step:27555 [D loss: 0.685328, acc: 59.38%] [G loss: 1.913472]
epoch:29 step:27556 [D loss: 0.618194, acc: 67.19%] [G loss: 1.822959]
epoch:29 step:27557 [D loss: 0.628523, acc: 67.19%] [G loss: 1.891041]
epoch:29 step:27558 [D loss: 0.590077, acc: 68.75%] [G loss: 2.009794]
epoch:29 step:27559 [D loss: 0.676094, acc: 59.38%] [G loss: 1.759941]
epoch:29 step:27560 [D loss: 0.660855, acc: 60.94%] [G loss: 1.734789]
epoch:29 step:27561 [D loss: 0.634823, acc: 71.09%] [G loss: 1.872915]
epoch:29 step:27562 [D loss: 0.678841, acc: 61.72%] [G loss: 1.959502]
epoch:29 step:27563 [D loss: 0.622918, acc: 66.41%] [G loss: 1.957385]
epoch:

epoch:29 step:27665 [D loss: 0.653095, acc: 61.72%] [G loss: 1.890800]
epoch:29 step:27666 [D loss: 0.697461, acc: 56.25%] [G loss: 1.879560]
epoch:29 step:27667 [D loss: 0.614205, acc: 65.62%] [G loss: 1.937566]
epoch:29 step:27668 [D loss: 0.604927, acc: 70.31%] [G loss: 1.970668]
epoch:29 step:27669 [D loss: 0.666480, acc: 56.25%] [G loss: 1.975945]
epoch:29 step:27670 [D loss: 0.581376, acc: 69.53%] [G loss: 2.081940]
epoch:29 step:27671 [D loss: 0.651828, acc: 64.06%] [G loss: 2.056845]
epoch:29 step:27672 [D loss: 0.621602, acc: 62.50%] [G loss: 2.162343]
epoch:29 step:27673 [D loss: 0.665321, acc: 58.59%] [G loss: 1.842368]
epoch:29 step:27674 [D loss: 0.675166, acc: 59.38%] [G loss: 1.770456]
epoch:29 step:27675 [D loss: 0.729036, acc: 51.56%] [G loss: 1.841109]
epoch:29 step:27676 [D loss: 0.665335, acc: 60.16%] [G loss: 1.886010]
epoch:29 step:27677 [D loss: 0.650941, acc: 64.84%] [G loss: 1.866985]
epoch:29 step:27678 [D loss: 0.661013, acc: 57.03%] [G loss: 1.896573]
epoch:

epoch:29 step:27785 [D loss: 0.608133, acc: 67.97%] [G loss: 1.965680]
epoch:29 step:27786 [D loss: 0.703209, acc: 57.81%] [G loss: 1.855056]
epoch:29 step:27787 [D loss: 0.688432, acc: 56.25%] [G loss: 1.827453]
epoch:29 step:27788 [D loss: 0.699960, acc: 52.34%] [G loss: 1.791870]
epoch:29 step:27789 [D loss: 0.684099, acc: 55.47%] [G loss: 1.818645]
epoch:29 step:27790 [D loss: 0.676000, acc: 60.94%] [G loss: 1.738539]
epoch:29 step:27791 [D loss: 0.652366, acc: 60.94%] [G loss: 1.918071]
epoch:29 step:27792 [D loss: 0.646927, acc: 67.19%] [G loss: 1.813283]
epoch:29 step:27793 [D loss: 0.625768, acc: 60.94%] [G loss: 1.795396]
epoch:29 step:27794 [D loss: 0.652920, acc: 63.28%] [G loss: 1.770909]
epoch:29 step:27795 [D loss: 0.670973, acc: 67.19%] [G loss: 1.891302]
epoch:29 step:27796 [D loss: 0.632710, acc: 64.06%] [G loss: 1.953150]
epoch:29 step:27797 [D loss: 0.616726, acc: 60.94%] [G loss: 2.050152]
epoch:29 step:27798 [D loss: 0.718976, acc: 51.56%] [G loss: 1.812433]
epoch:

epoch:29 step:27900 [D loss: 0.671824, acc: 62.50%] [G loss: 1.850643]
epoch:29 step:27901 [D loss: 0.669560, acc: 60.94%] [G loss: 1.858865]
epoch:29 step:27902 [D loss: 0.682136, acc: 57.81%] [G loss: 1.823328]
epoch:29 step:27903 [D loss: 0.649981, acc: 62.50%] [G loss: 1.767859]
epoch:29 step:27904 [D loss: 0.622555, acc: 64.84%] [G loss: 1.831787]
epoch:29 step:27905 [D loss: 0.665841, acc: 56.25%] [G loss: 1.796183]
epoch:29 step:27906 [D loss: 0.623570, acc: 61.72%] [G loss: 2.004740]
epoch:29 step:27907 [D loss: 0.657718, acc: 60.16%] [G loss: 1.693917]
epoch:29 step:27908 [D loss: 0.698374, acc: 53.91%] [G loss: 1.964904]
epoch:29 step:27909 [D loss: 0.592635, acc: 69.53%] [G loss: 1.986781]
epoch:29 step:27910 [D loss: 0.598863, acc: 69.53%] [G loss: 1.858143]
epoch:29 step:27911 [D loss: 0.731234, acc: 54.69%] [G loss: 1.843491]
epoch:29 step:27912 [D loss: 0.639030, acc: 60.94%] [G loss: 1.894652]
epoch:29 step:27913 [D loss: 0.650037, acc: 55.47%] [G loss: 2.007142]
epoch:

epoch:29 step:28015 [D loss: 0.624531, acc: 63.28%] [G loss: 1.898288]
epoch:29 step:28016 [D loss: 0.655608, acc: 61.72%] [G loss: 1.924232]
epoch:29 step:28017 [D loss: 0.657845, acc: 57.81%] [G loss: 1.914780]
epoch:29 step:28018 [D loss: 0.628602, acc: 60.16%] [G loss: 1.876128]
epoch:29 step:28019 [D loss: 0.678655, acc: 61.72%] [G loss: 1.828905]
epoch:29 step:28020 [D loss: 0.589605, acc: 66.41%] [G loss: 1.866910]
epoch:29 step:28021 [D loss: 0.639463, acc: 64.06%] [G loss: 1.747290]
epoch:29 step:28022 [D loss: 0.618296, acc: 64.06%] [G loss: 1.890716]
epoch:29 step:28023 [D loss: 0.689719, acc: 52.34%] [G loss: 1.620619]
epoch:29 step:28024 [D loss: 0.685382, acc: 57.81%] [G loss: 1.807664]
epoch:29 step:28025 [D loss: 0.693895, acc: 56.25%] [G loss: 1.968593]
epoch:29 step:28026 [D loss: 0.758088, acc: 46.88%] [G loss: 1.878789]
epoch:29 step:28027 [D loss: 0.653259, acc: 58.59%] [G loss: 1.855421]
epoch:29 step:28028 [D loss: 0.699560, acc: 54.69%] [G loss: 1.663760]
epoch: